In [171]:
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import multiprocessing as mp
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf



In [172]:
df_meta = pd.read_csv("./birdclef-2024/train_metadata.csv")

In [173]:
df_meta

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg
1,asbfly,[],['song'],51.4030,104.6401,Muscicapa dauurica,Asian Brown Flycatcher,Magnus Hellström,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/164848,asbfly/XC164848.ogg
2,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg
3,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg
...,...,...,...,...,...,...,...,...,...,...,...,...
24454,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://xeno-canto.org/845747,zitcis1/XC845747.ogg
24455,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/845817,zitcis1/XC845817.ogg
24456,zitcis1,[],[''],51.1207,4.5607,Cisticola juncidis,Zitting Cisticola,Wim Jacobs,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/856176,zitcis1/XC856176.ogg
24457,zitcis1,[],[''],41.5607,-8.4236,Cisticola juncidis,Zitting Cisticola,Jorge Leitão,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://xeno-canto.org/856723,zitcis1/XC856723.ogg


In [174]:
df_train = df_meta[["primary_label", "filename"]]

In [175]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

    

In [176]:
def extract_audio_features_with_path(args):
    ogg_file_path, max_length = args
    return extract_audio_features(ogg_file_path, max_length)

def extract_audio_features(ogg_file_path, max_length=22050*5):
    y, sr = librosa.load(ogg_file_path, sr=None)
    
    # Ensure the audio is of fixed length
    if len(y) < max_length:
        y = np.pad(y, (0, max_length - len(y)), 'constant')
    else:
        y = y[:max_length]
    
    # Extract features
    features = {}

    # Mel spectrogram
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    S_dB = librosa.power_to_db(S, ref=np.max)
    features['mel_spectrogram'] = S_dB

    # MFCC
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    features['mfcc'] = mfcc

    # Chroma feature
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    features['chroma'] = chroma

    # Spectral contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    features['spectral_contrast'] = spectral_contrast

    # Tonnetz
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
    features['tonnetz'] = tonnetz

    # Spectral centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    features['spectral_centroid'] = spectral_centroid

    # Spectral bandwidth
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    features['spectral_bandwidth'] = spectral_bandwidth

    # Spectral rolloff
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    features['spectral_rolloff'] = spectral_rolloff

    # Zero crossing rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
    features['zero_crossing_rate'] = zero_crossing_rate

    # RMS
    rms = librosa.feature.rms(y=y)
    features['rms'] = rms

    return features


In [177]:

# Example usage
audio_features = extract_audio_features('./birdclef-2024/train_audio/asbfly/XC49755.ogg')

# Convert features to a dictionary of DataFrames for better visualization
features_df = {key: pd.DataFrame(value) for key, value in audio_features.items()}

# Display the extracted features
for feature_name, df in features_df.items():
    print(f"\nFeature: {feature_name}")
    display(df.head())  # Using display() from IPython.display for better visualization in Jupyter


Feature: mel_spectrogram


,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,-80.0,-80.0,-80.0,-80.0,-71.131088,-64.629059,-64.798874,-65.517334,-64.943108,-66.595901,...,-68.542099,-62.939037,-61.753532,-64.483032,-63.943806,-68.828239,-75.564331,-80.000000,-69.745407,-58.308243
1,-80.0,-80.0,-80.0,-80.0,-78.152908,-77.862320,-80.000000,-80.000000,-80.000000,-80.000000,...,-66.897888,-64.690094,-63.297081,-61.861931,-62.233372,-65.241409,-66.423172,-74.840057,-68.645630,-57.374527
2,-80.0,-80.0,-80.0,-80.0,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,...,-61.738167,-63.307991,-63.542484,-68.342438,-66.267380,-65.165840,-68.677383,-73.797440,-69.288124,-58.136021
3,-80.0,-80.0,-80.0,-80.0,-78.352684,-80.000000,-80.000000,-80.000000,-80.000000,-79.861069,...,-64.366806,-71.845596,-70.448761,-73.537865,-71.473488,-71.168457,-72.105133,-69.305962,-67.361351,-57.354900
4,-80.0,-80.0,-80.0,-80.0,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,...,-73.917877,-80.000000,-76.070702,-72.207718,-73.334587,-75.337097,-80.000000,-70.599457,-68.497894,-58.325111



Feature: mfcc


,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,-695.373413,-695.373413,-695.354370,-680.230347,-643.505615,-619.225769,-591.797607,-570.970276,-563.205383,-558.302429,...,-306.195435,-323.671509,-334.408752,-339.167542,-338.049377,-331.991241,-328.520203,-321.373688,-314.663330,-300.228363
1,0.000000,0.000000,0.000993,-1.203629,-5.267550,-6.745503,-6.516576,-5.824796,-6.088685,-6.661308,...,-41.371971,-18.021278,-13.957634,-14.097694,-14.342091,-13.824339,-19.953579,-27.281540,-24.620070,-22.640427
2,0.000000,0.000000,-0.026895,-17.487478,-48.594490,-65.481277,-84.124863,-95.908478,-98.228439,-101.852119,...,-174.743820,-175.672653,-179.184845,-179.671799,-181.474518,-182.583755,-186.912918,-188.352631,-176.718719,-133.417725
3,0.000000,0.000000,-0.002972,2.805939,12.572544,14.241900,11.571678,11.357302,12.281720,13.987471,...,65.480560,54.011566,45.415138,47.022881,52.940201,50.364639,52.326851,65.001930,67.655037,71.527222
4,0.000000,0.000000,0.026674,8.966082,11.528899,8.192680,3.498781,-2.270909,-9.443661,-8.712378,...,-91.412788,-79.573349,-68.684563,-62.239914,-48.788330,-53.536591,-59.000977,-60.851517,-54.280037,-25.470657



Feature: chroma


...

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,13000.0,12156.25,10078.125,7750.0,6421.875,6343.75,6312.5,6078.125,6203.125,6015.625,...,6109.375,6093.75,5968.75,5796.875,5703.125,5515.625,5375.0,5375.0,5609.375,6031.25



Feature: zero_crossing_rate


,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,0.223633,0.297852,0.361328,0.295898,0.250488,0.234863,0.235352,0.245117,0.246582,0.242676,...,0.323242,0.3125,0.288086,0.258301,0.239258,0.218262,0.212891,0.219727,0.186035,0.133789



Feature: rms


,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,0.000013,0.000031,0.000073,0.000177,0.00027,0.000456,0.000616,0.000727,0.000869,0.000953,...,0.090907,0.066837,0.030106,0.029994,0.035177,0.04654,0.061427,0.067807,0.066481,0.057839


In [178]:
def aggregate_features(features):
    aggregated_features = {}
    for key, value in features.items():
        aggregated_features[key] = {
            'mean': np.mean(value, axis=1),
            'std': np.std(value, axis=1),
            'min': np.min(value, axis=1),
            'max': np.max(value, axis=1)
        }
    return aggregated_features

def format_features(aggregated_features):
    formatted_features = []
    for key in aggregated_features:
        for stat in aggregated_features[key]:
            formatted_features.extend(aggregated_features[key][stat])
    return np.array(formatted_features)

In [179]:
ogg_file_path = './birdclef-2024/train_audio/asbfly/XC49755.ogg'
max_length = 22050 * 5  # For example, 5 seconds at a sample rate of 22050 Hz

# Extract and aggregate features
features = extract_audio_features(ogg_file_path, max_length)
aggregated_features = aggregate_features(features)

# Format features for model input
formatted_features = format_features(aggregated_features)

print(formatted_features.shape)
print(formatted_features)


(684,)
[-6.78951492e+01 -7.15780563e+01 -7.08037796e+01 -7.23653183e+01
 -7.57353897e+01 -7.60923386e+01 -7.72794800e+01 -7.69549332e+01
 -7.69534836e+01 -7.48166885e+01 -7.59738770e+01 -7.50231247e+01
 -7.48816299e+01 -7.21270599e+01 -7.18847809e+01 -6.99784622e+01
 -6.92440186e+01 -6.64677048e+01 -6.55641174e+01 -6.29078369e+01
 -6.35348587e+01 -6.19041214e+01 -6.14968338e+01 -6.00429764e+01
 -5.98880310e+01 -5.87049522e+01 -5.84747009e+01 -5.72715302e+01
 -5.75240135e+01 -5.58471909e+01 -5.55498390e+01 -5.40977554e+01
 -5.38592224e+01 -5.26902885e+01 -5.27443390e+01 -5.14852295e+01
 -5.19623528e+01 -5.10422821e+01 -5.14217949e+01 -5.06474037e+01
 -5.04771423e+01 -4.96308174e+01 -5.02442932e+01 -4.85827980e+01
 -4.86090584e+01 -4.86892662e+01 -4.79537125e+01 -4.74637413e+01
 -4.74799538e+01 -4.70840149e+01 -4.66258774e+01 -4.66038132e+01
 -4.59247398e+01 -4.57369804e+01 -4.46376724e+01 -4.42834740e+01
 -4.41352196e+01 -4.43094559e+01 -4.40298653e+01 -4.37026901e+01
 -4.32755165e+01 -

In [180]:

def load_data(file_paths, labels, max_length=22050*5):
    X = []
    y = []

    for file_path, label in zip(file_paths, labels):
        features = extract_audio_features(file_path, max_length)
        aggregated_features = aggregate_features(features)
        formatted_features = format_features(aggregated_features)
        X.append(formatted_features)
        y.append(label)
        # print(len(X), len(X[-1]),y)
        if len(X) % 100 == 0:
            print(f"Processed {len(X)} files")
    return np.array(X), np.array(y)


In [181]:
import jupyterlab
print(jupyterlab.__version__)

4.2.1


In [182]:
# Assuming df_train is already loaded in your notebook
file_paths = df_train['filename'].apply(lambda x : "./birdclef-2024/train_audio/" + x ).tolist()
labels = df_train['primary_label'].astype('category').cat.codes.tolist()

# Load data
# X, y = load_data(file_paths, labels)
X, y = np.load('X.npy'), np.load('y.npy')# load_data(file_paths, labels)

In [183]:
# save the x and y

np.save('X.npy', X)
np.save('y.npy', y)

In [184]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

def extract_and_format_features(file_path, max_length=22050*5):
    try:
        features = extract_audio_features(file_path, max_length)
        aggregated_features = aggregate_features(features)
        formatted_features = format_features(aggregated_features)
        return formatted_features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def process_batch(file_paths, labels, max_length):
    X_batch = []
    y_batch = []
    for file_path, label in zip(file_paths, labels):
        formatted_features = extract_and_format_features(file_path, max_length)
        if formatted_features is not None:
            X_batch.append(formatted_features)
            y_batch.append(label)
    return X_batch, y_batch

def load_data(file_paths, labels, max_length=22050*5, batch_size=1000):
    X = []
    y = []

    # Split the data into batches
    batches = [(file_paths[i:i + batch_size], labels[i:i + batch_size])
               for i in range(0, len(file_paths), batch_size)]

    # Process batches in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_batch, batch_files, batch_labels, max_length) 
                   for batch_files, batch_labels in batches]
        
        for future in futures:
            X_batch, y_batch = future.result()
            X.extend(X_batch)
            y.extend(y_batch)

    return np.array(X), np.array(y)

# Assuming df_train is already loaded in your notebook
file_paths = df_train['filename'].apply(lambda x: "./birdclef-2024/train_audio/" + x).tolist()
labels = df_train['primary_label'].astype('category').cat.codes.tolist()

# Load data
X, y = np.load('X.npy'), np.load('y.npy')# load_data(file_paths, labels)

print(f"Loaded {X.shape[0]} files.")
print(X.shape)
print(y.shape)


Loaded 24459 files.
(24459, 684)
(24459,)


In [185]:
len(df_train)

24459

In [186]:
# save x and y as csv

# np.savetxt('X.csv', X, delimiter=',')
# np.savetxt('y.csv', y, delimiter=',')
print(X.shape)
print(y.shape)


(24459, 684)
(24459,)


In [187]:
print(X[1])

[-1.58963785e+01 -1.62785816e+01 -1.60039845e+01 -1.61279907e+01
 -1.68902264e+01 -1.66303806e+01 -1.90093765e+01 -1.82996597e+01
 -1.97097206e+01 -1.88165798e+01 -1.93298340e+01 -1.86974010e+01
 -1.96464577e+01 -1.79825916e+01 -1.93120747e+01 -1.85301857e+01
 -1.84365807e+01 -1.69842072e+01 -1.81452885e+01 -1.72752266e+01
 -1.80543804e+01 -1.79443626e+01 -1.84281940e+01 -1.83274288e+01
 -1.93307648e+01 -1.77445602e+01 -1.90370407e+01 -1.87021999e+01
 -1.97975559e+01 -1.95816879e+01 -2.00255089e+01 -1.96887741e+01
 -2.06255875e+01 -2.05158596e+01 -2.18840008e+01 -2.18313236e+01
 -2.27161179e+01 -2.29569702e+01 -2.43032150e+01 -2.38553371e+01
 -2.37021084e+01 -2.30606480e+01 -2.36355991e+01 -2.39111691e+01
 -2.52166252e+01 -2.51034756e+01 -2.48021603e+01 -2.50561848e+01
 -2.54581928e+01 -2.63199348e+01 -2.61479130e+01 -2.68570290e+01
 -2.75378208e+01 -2.76385975e+01 -2.76007042e+01 -2.82462063e+01
 -2.88554058e+01 -2.94312649e+01 -2.99256134e+01 -2.95353432e+01
 -3.00814590e+01 -2.95706

In [188]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert labels to categorical
y = to_categorical(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X.shape, y.shape)
# Define the neural network model

def create_model(input_shape):
    model = Sequential([
        Dense(256, input_shape=(input_shape,), activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dense(y_train.shape[1], activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the model
input_shape = X_train.shape[1]
model = create_model(input_shape)

# Define early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=100, restore_best_weights=True)

# Define model checkpointing
checkpoint = ModelCheckpoint(
    filepath='best_model.weights.h5',  # Path where the model weights will be saved
    monitor='val_accuracy',  # Metric to monitor
    save_best_only=True,  # Only save the model if the metric improves
    mode='max',  # Minimize the monitored metric
    save_weights_only=True,  # Only save the model's weights
    verbose=1  # Print a message when saving the model
)

# Train the model with early stopping and model checkpointing
history = model.fit(
    X_train, y_train,
    epochs=800,
    batch_size=256,
    validation_split=0.2,
    callbacks=[early_stopping, checkpoint]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# Load the best weights
model.load_weights('best_model.weights.h5')

# Re-evaluate the model with the best weights
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy with best weights: {accuracy:.4f}")


(24459, 684) (24459, 182)
Epoch 1/800


/home/rel-wazzi/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


56/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0151 - loss: 5.9118
Epoch 1: val_accuracy improved from -inf to 0.09121, saving model to best_model.weights.h5
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.0170 - loss: 5.8908 - val_accuracy: 0.0912 - val_loss: 5.2230
Epoch 2/800
53/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0803 - loss: 5.1953
Epoch 2: val_accuracy improved from 0.09121 to 0.15457, saving model to best_model.weights.h5
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0830 - loss: 5.1687 - val_accuracy: 0.1546 - val_loss: 4.5990
Epoch 3/800
60/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1415 - loss: 4.5849
Epoch 3: val_accuracy improved from 0.15457 to 0.18855, saving model to best_model.weights.h5
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1417 - loss: 4.5814 - val_accuracy: 0.1886 - val_loss: 4.2173
Epoch 4/800
57/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1647 - loss: 4.2900
Epoch 4: val_accuracy improved from 0.18855 to 0

In [190]:
# save the model as an image

from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)


You must install pydot (`pip install pydot`) for `plot_model` to work.
